In [1]:
import random
import math
import copy
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson as poi
from scipy.stats import expon

In [2]:
numClass = 3
lambdas = [0, 0.5, 0.5, 0.5]

rho = [[],
       [0.8, 0, 0.18, 0.02],
       [0.3, 0.6, 0, 0.1],
       [0.1, 0.2, 0.7, 0]]

alpha = [0, 1/10, 1/40, 1/80]

re1 = 1
re2 = 3
re3 = 4

co1 = 5
co2 = 3
co3 = 2

In [3]:
class patient():
    def __init__(self, initClass, arrTime, currentClass, classTime, clockTime, curedClass, diedClass, endTime):
        self.initClass = initClass
        self.arrTime = arrTime
        self.currentClass = currentClass
        self.classTime = classTime
        self.clockTime = clockTime
        self.curedClass = curedClass
        self.diedClass = diedClass
        self.endTime = endTime

    def worseTime(self):
        i = 1
        worseT = 0
        while i <= self.currentClass:
            worseT += self.classTime[i]
            i += 1
        return worseT

In [4]:
def newPatient(inputClass, T):
    initClass = inputClass
    currentClass = initClass
    diedClass = currentClass
    classTime = [0 for i in range(numClass + 1)]
    clockTime = math.ceil(expon.rvs(scale = 1/alpha[inputClass]))
    return patient(initClass, T, currentClass, classTime, clockTime, 0, diedClass, 0)

In [5]:
def inputProcess(numClass, rho):
    classes = [i for i in range(numClass+1)]
    patientsList = []
    rhoC = copy.deepcopy(rho)
    for row in rhoC[1:]:
        for j in range(1, len(row)):
            row[j] = row[j] + row[j-1]

    return classes, patientsList, rhoC

In [6]:
def newArr(classes, patientsList, lambdas, alpha, T):
    for i in classes[1:]:
        rnum = poi.rvs(lambdas[i])
        #print('newArr:', rnum)
        for j in range(rnum):
            patientsList.append(newPatient(i, T))
    return patientsList

In [7]:
def pUpdate(classes, patientsList, alpha, rhoC, T):
    if len(patientsList) > 0:
        for p in patientsList:
            if p.currentClass != -1 and p.currentClass != 0:
                p.classTime[p.currentClass] +=1
                p.clockTime -=1

                if p.clockTime == 0:
                    rnum = random.random()
                    for j in range(len(rhoC[p.currentClass])):
                        if rnum <= rhoC[p.currentClass][j]:
                            p.currentClass = j
                            break

                    if p.currentClass != 0:
                        p.clockTime = math.ceil(expon.rvs(1/alpha[p.currentClass]))
                        p.diedClass = p.currentClass
                    if p.currentClass == 0:
                        p.endTime = T
                    
    return patientsList

In [8]:
def policy(patientsList, classOrder):
    minClass = len(classOrder)
    maxWorseT = 0
    c = False
    for i in range(len(patientsList)):
        if patientsList[i].currentClass > 0:
            if classOrder.index(patientsList[i].currentClass) < minClass:
                minClass = classOrder.index(patientsList[i].currentClass)
                c = i
            if classOrder.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
                maxWorseT = patientsList[i].worseTime()
                c = i
    return c

# def policy2(patientsList):
#     classes_mod = [1, 3, 2]
#     minClass = len(classes_mod)
#     maxWorseT = 0
#     c = False
#     for i in range(len(patientsList)):
#         if patientsList[i].currentClass > 0:
#             if classes_mod.index(patientsList[i].currentClass) < minClass:
#                 minClass = classes_mod.index(patientsList[i].currentClass)
#                 c = i
#             if classes_mod.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
#                 maxWorseT = patientsList[i].worseTime()
#                 c = i
#     return c

# def policy3(patientsList):
#     classes_mod = [2, 1, 3]
#     minClass = len(classes_mod)
#     maxWorseT = 0
#     c = False
#     for i in range(len(patientsList)):
#         if patientsList[i].currentClass > 0:
#             if classes_mod.index(patientsList[i].currentClass) < minClass:
#                 minClass = classes_mod.index(patientsList[i].currentClass)
#                 c = i
#             if classes_mod.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
#                 maxWorseT = patientsList[i].worseTime()
#                 c = i
#     return c

# def policy4(patientsList):
#     classes_mod = [2, 3, 1]
#     minClass = len(classes_mod)
#     maxWorseT = 0
#     c = False
#     for i in range(len(patientsList)):
#         if patientsList[i].currentClass > 0:
#             if classes_mod.index(patientsList[i].currentClass) < minClass:
#                 minClass = classes_mod.index(patientsList[i].currentClass)
#                 c = i
#             if classes_mod.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
#                 maxWorseT = patientsList[i].worseTime()
#                 c = i
#     return c

# def policy5(patientsList):
#     classes_mod = [3, 1, 2]
#     minClass = len(classes_mod)
#     maxWorseT = 0
#     c = False
#     for i in range(len(patientsList)):
#         if patientsList[i].currentClass > 0:
#             if classes_mod.index(patientsList[i].currentClass) < minClass:
#                 minClass = classes_mod.index(patientsList[i].currentClass)
#                 c = i
#             if classes_mod.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
#                 maxWorseT = patientsList[i].worseTime()
#                 c = i
#     return c

# def policy6(patientsList):
#     classes_mod = [3, 2, 1]
#     minClass = len(classes_mod)
#     maxWorseT = 0
#     c = False
#     for i in range(len(patientsList)):
#         if patientsList[i].currentClass > 0:
#             if classes_mod.index(patientsList[i].currentClass) < minClass:
#                 minClass = classes_mod.index(patientsList[i].currentClass)
#                 c = i
#             if classes_mod.index(patientsList[i].currentClass) <= minClass and patientsList[i].worseTime() > maxWorseT:
#                 maxWorseT = patientsList[i].worseTime()
#                 c = i
#     return c

In [9]:
def serv(classes, patientsList, mu, T, classOrder):
    rnum = poi.rvs(mu)
    
    for j in range(rnum):
        c = policy(patientsList, classOrder)
        if c:
            patientsList[c].curedClass = patientsList[c].currentClass
            patientsList[c].currentClass = -1
            patientsList[c].endTime = T
    return patientsList

# def serv2(classes, patientsList, mu, T):
#     rnum = poi.rvs(mu)
    
#     for j in range(rnum):
#         c = policy2(patientsList)
#         if c:
#             patientsList[c].curedClass = patientsList[c].currentClass
#             patientsList[c].currentClass = -1
#             patientsList[c].endTime = T
#     return patientsList

# def serv3(classes, patientsList, mu, T):
#     rnum = poi.rvs(mu)
    
#     for j in range(rnum):
#         c = policy3(patientsList)
#         if c:
#             patientsList[c].curedClass = patientsList[c].currentClass
#             patientsList[c].currentClass = -1
#             patientsList[c].endTime = T
#     return patientsList

# def serv4(classes, patientsList, mu, T):
#     rnum = poi.rvs(mu)
    
#     for j in range(rnum):
#         c = policy4(patientsList)
#         if c:
#             patientsList[c].curedClass = patientsList[c].currentClass
#             patientsList[c].currentClass = -1
#             patientsList[c].endTime = T
#     return patientsList

# def serv5(classes, patientsList, mu, T):
#     rnum = poi.rvs(mu)
    
#     for j in range(rnum):
#         c = policy5(patientsList)
#         if c:
#             patientsList[c].curedClass = patientsList[c].currentClass
#             patientsList[c].currentClass = -1
#             patientsList[c].endTime = T
#     return patientsList

# def serv6(classes, patientsList, mu, T):
#     rnum = poi.rvs(mu)
    
#     for j in range(rnum):
#         c = policy6(patientsList)
#         if c:
#             patientsList[c].curedClass = patientsList[c].currentClass
#             patientsList[c].currentClass = -1
#             patientsList[c].endTime = T
#     return patientsList


In [10]:
def mainSim(T_end, numClass, lambdas, mu, rho, alpha, classOrder):
    classes, patientsList, rhoC = inputProcess(numClass, rho)
    T = 0
    servList = []
    while T <= T_end:
        # update patients status
        patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)
         
        # check new arrivals
        patientsList = newArr(classes, patientsList, lambdas, alpha, T)     
   
        # check service
        patientsList = serv(classes, patientsList, mu, T, classOrder)
        
        T += 1
    return patientsList

# def mainSim2(T_end, numClass, lambdas, mu, rho, alpha):
#     classes, patientsList, rhoC = inputProcess(numClass, rho)
#     T = 0
#     servList = []
#     while T <= T_end:
#         # update patients status
#         patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)

#         # check new arrivals
#         patientsList = newArr(classes, patientsList, lambdas, alpha, T)     

#         # check service
#         patientsList = serv2(classes, patientsList, mu, T)

#         T += 1
#     return patientsList

# def mainSim3(T_end, numClass, lambdas, mu, rho, alpha):
#     classes, patientsList, rhoC = inputProcess(numClass, rho)
#     T = 0
#     servList = []
#     while T <= T_end:
#         # update patients status
#         patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)

#         # check new arrivals
#         patientsList = newArr(classes, patientsList, lambdas, alpha, T)     

#         # check service
#         patientsList = serv3(classes, patientsList, mu, T)

#         T += 1
#     return patientsList

# def mainSim4(T_end, numClass, lambdas, mu, rho, alpha):
#     classes, patientsList, rhoC = inputProcess(numClass, rho)
#     T = 0
#     servList = []
#     while T <= T_end:
#         # update patients status
#         patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)

#         # check new arrivals
#         patientsList = newArr(classes, patientsList, lambdas, alpha, T)     

#         # check service
#         patientsList = serv4(classes, patientsList, mu, T)

#         T += 1
#     return patientsList

# def mainSim5(T_end, numClass, lambdas, mu, rho, alpha):
#     classes, patientsList, rhoC = inputProcess(numClass, rho)
#     T = 0
#     servList = []
#     while T <= T_end:
#         # update patients status
#         patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)

#         # check new arrivals
#         patientsList = newArr(classes, patientsList, lambdas, alpha, T)     

#         # check service
#         patientsList = serv5(classes, patientsList, mu, T)

#         T += 1
#     return patientsList

# def mainSim6(T_end, numClass, lambdas, mu, rho, alpha):
#     classes, patientsList, rhoC = inputProcess(numClass, rho)
#     T = 0
#     servList = []
#     while T <= T_end:
#         # update patients status
#         patientsList = pUpdate(classes, patientsList, alpha, rhoC, T)

#         # check new arrivals
#         patientsList = newArr(classes, patientsList, lambdas, alpha, T)     

#         # check service
#         patientsList = serv6(classes, patientsList, mu, T)

#         T += 1
#     return patientsList

In [80]:
mu = 15
orders = [[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]

abd = []
totalR = []
totalC = []
c1 = []
c2 = []
c3 = []
lp = []

for i in range(10):
    t_end = 5000
    
    abds = []
    totalRs = []
    totalCs = []
    c1s = []
    c2s = []
    c3s = []
    lps = []
    for classOrder in orders:
        pp = mainSim(t_end, numClass, lambdas, mu, rho, alpha, classOrder)
        
        abdnum = 0 
        R = 0
        C = 0
        c1c = 0
        c2c = 0
        c3c = 0
        lpnum = len(pp)
        for p in pp:
            if p.currentClass == 0:
                abdnum +=1
            if p.currentClass == -1:
                if p.curedClass == 1:
                    R += re1
                    c1c +=1
                if p.curedClass == 2:
                    R += re2
                    c2c +=1
                if p.curedClass == 3:
                    R += re3
                    c3c +=1
            C += co1*p.classTime[1]+co2*p.classTime[2]+co3*p.classTime[3]
        
        abds.append(abdnum)
        totalRs.append(R)
        totalCs.append(C)
        c1s.append(c1c)
        c2s.append(c2c)
        c3s.append(c3c)
        lps.append(lpnum)
        
    abd.append(abds)
    totalR.append(totalRs)
    totalC.append(totalCs)
    c1.append(c1s)
    c2.append(c2s)
    c3.append(c3s)
    lp.append(lps)
    print(i)

0
1
2
3
4
5
6
7
8
9


In [81]:
np.savetxt("abd_"+str(mu)+".csv", abd, delimiter=",")

np.savetxt("totalR_"+str(mu)+".csv", totalR, delimiter=",")

np.savetxt("totalC_"+str(mu)+".csv", totalC, delimiter=",")

np.savetxt("c1_"+str(mu)+".csv", c1, delimiter=",")

np.savetxt("c2_"+str(mu)+".csv", c2, delimiter=",")

np.savetxt("c3_"+str(mu)+".csv", c3, delimiter=",")

np.savetxt("lp_"+str(mu)+".csv", lp, delimiter=",")

In [82]:
abd_rate = np.asarray(abd)/np.asarray(lp)
totalR_ary = np.asarray(totalR)
totalC_ary = np.asarray(totalC)

rho_alpha = []
for i in range(5):
    a = abd_rate[:,0]
    b = abd_rate[:,i+1]
    t_score, p_value = sp.stats.ttest_ind(a, b, axis=0, equal_var=False)
    rho_alpha.append(orders[i+1]+[t_score, p_value])
    
R_rule = []
for i in range(5):
    a = totalR_ary[:,i]
    b = totalR_ary[:,5]
    t_score, p_value = sp.stats.ttest_ind(a, b, axis=0, equal_var=False)
    R_rule.append(orders[i]+[t_score, p_value])
    
R_alpha = []
for i in range(5):
    a = totalR_ary[:,i+1]
    b = totalR_ary[:,0]
    t_score, p_value = sp.stats.ttest_ind(a, b, axis=0, equal_var=False)
    R_alpha.append(orders[i+1]+[t_score, p_value])
    
C_rule = []
for i in range(5):
    a = totalC_ary[:,0]
    b = totalC_ary[:,i+1]
    t_score, p_value = sp.stats.ttest_ind(a, b, axis=0, equal_var=False)
    C_rule.append(orders[i+1]+[t_score, p_value])
    
C_alpha = []
for i in range(5):
    a = totalC_ary[:,5]
    b = totalC_ary[:,i]
    t_score, p_value = sp.stats.ttest_ind(a, b, axis=0, equal_var=False)
    C_alpha.append(orders[i]+[t_score, p_value])
    
np.savetxt("rho_alpha_"+str(mu)+".csv", rho_alpha, delimiter=",")
np.savetxt("R_rule_"+str(mu)+".csv", R_rule, delimiter=",")
np.savetxt("R_alpha_"+str(mu)+".csv", R_alpha, delimiter=",")
np.savetxt("C_rule_"+str(mu)+".csv", C_rule, delimiter=",")
np.savetxt("C_alpha_"+str(mu)+".csv", C_alpha, delimiter=",")

In [60]:
rho_alpha

[[1, 3, 2, -1.4378569575349396, 0.1682677423851148],
 [2, 1, 3, -1.7383671663918598, 0.099354371882100584],
 [2, 3, 1, -1.7824209858323581, 0.091697085690178509],
 [3, 1, 2, -3.1532770103260179, 0.0064289946394259969],
 [3, 2, 1, -1.8171137182468935, 0.089557677123840884]]

In [61]:
R_rule

[[1, 2, 3, -184.67703665270034, 1.4923668918457036e-29],
 [1, 3, 2, -201.36520251092298, 1.9710402565004349e-31],
 [2, 1, 3, -41.233949526751182, 5.9849228293904751e-16],
 [2, 3, 1, -39.64652114820332, 7.6920216465374169e-16],
 [3, 1, 2, -24.465352529424557, 4.3627611501932227e-12]]

In [62]:
R_alpha

[[1, 3, 2, 0.79049340446083649, 0.43977513594508499],
 [2, 1, 3, 95.477573047482579, 1.4703669787971587e-23],
 [2, 3, 1, 98.690146638263016, 5.5862309571481947e-24],
 [3, 1, 2, 96.766280793419767, 5.4899254564506335e-22],
 [3, 2, 1, 184.67703665270034, 1.4923668918457036e-29]]

In [63]:
C_rule

[[1, 3, 2, 0.01775101530257046, 0.98603317230247189],
 [2, 1, 3, 24.777392241742159, 1.0938496105219951e-14],
 [2, 3, 1, 24.746896935374966, 3.4536846904062394e-14],
 [3, 1, 2, 60.476960507740699, 3.2303770522506829e-19],
 [3, 2, 1, 74.511878878885241, 6.1832719997195737e-18]]

In [64]:
C_alpha

[[1, 2, 3, -74.511878878885241, 6.1832719997195737e-18],
 [1, 3, 2, -71.6822584588799, 2.0786346591593402e-17],
 [2, 1, 3, -29.561666874295646, 6.5835291201367166e-12],
 [2, 3, 1, -25.249743719439234, 6.676650675280877e-11],
 [3, 1, 2, -14.344894531474814, 9.1590292093095331e-11]]

In [46]:
orders

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]